In [2]:
!pip install faiss-cpu langchain pandas pypdf langchain-openai openpyxl langchain-ollama

In [3]:
import os
os.environ["OPENAI_API_KEY"] = "sk" #openai 키 입력

from langchain_ollama import OllamaLLM
# DeepSeek R1 모델 로드
deepseek = OllamaLLM(model="deepseek-r1")

In [4]:
from langchain.document_loaders import PyPDFLoader
docs = [] # 추출된 문서를 저장할 리스트를 초기화

loader = PyPDFLoader("d:/data/succulent.pdf") # "d:/data/succulent.pdf" 경로의 PDF 파일을 PyPDFLoader로 로드

# PDF에서 텍스트 데이터를 추출하여 텍스트 데이터 반환
text_content = loader.load() 

# PDF의 각 페이지를 하나씩 가져와 docs에 저장
for doc in text_content:
    docs.append(doc)
print(docs[0].page_content[:100]) # 첫 번째 페이지의 처음 100자 출력
print(docs[0].metadata) # 첫 번째 문서(첫 페이지)의 메타데이터 출력

🌵 다육이 키우는 방법 
1. 햇빛과 온도 관리 
• 햇빛: 다육이는 햇빛을 좋아하는 식물이므로 하루 4~6 시간 이상은 직사광선을 받게 
해주세요. 
o 창가, 베란다, 야외 등
{'source': 'd:/data/succulent.pdf', 'page': 0, 'page_label': '1'}


In [5]:
from langchain_openai import OpenAIEmbeddings
from langchain import FAISS # Facebook에서 개발한 벡터 검색을 위한 Similarity Search 라이브러리
from langchain.text_splitter import CharacterTextSplitter

# 문서 분할 (텍스트를 작은 단위로 나누기)
text_splitter = CharacterTextSplitter(
    separator="\n\n", # 두 줄 띄움(\n\n)을 기준으로 분할
    chunk_size=100, # 긴 텍스트를 100자 단위로 분할
    chunk_overlap=10, # 각 청크 사이에 10자 겹치도록 설정
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(text_content) # text_content에서 텍스트를 가져와 분할
# 문서를 FAISS가 처리할 수 있도록, 텍스트(page_content)만 추출해서 texts에 저장
texts = [doc.page_content for doc in splits]

# OpenAI 임베딩 모델 로드
embeddings = OpenAIEmbeddings()

# 변환된 벡터를 FAISS 벡터 데이터베이스에 저장
vectorstore = FAISS.from_texts(texts, embeddings)

# FAISS 벡터 인덱스를 로컬 파일(d:/data/db_faiss_complete_open_ai)에 저장
vectorstore.save_local('d:/data/db_faiss_complete_open_ai')

In [6]:
from langchain_core.prompts import PromptTemplate
# PromptTemplate를 사용하여 맞춤형 프롬프트 생성
custom_prompt_template = PromptTemplate(
    template = """당신은 다육이를 키우는 정보를 제공하는 챗봇입니다.

{context}

제공된 소스 데이터만 사용하여 고객의 질문에 답하세요.
모르겠다면 "잘 모르겠습니다."라고 답하세요.
친절하고 예의 바르며 전문적인 어조를 사용하세요.
답변은 간결하게 유지하세요.

질문: {question}

답변:
""",
input_variables=["context"]
)
chain_type_kwargs = {"prompt": custom_prompt_template} # 프롬프트를 랭체인 체인(chain_type_kwargs)에 전달

In [7]:
# FAISS 데이터베이스 로드 (FAISS.load_local)
retriever = FAISS.load_local(
    "d:/data/db_faiss_complete_open_ai", 
    OpenAIEmbeddings(),  # 저장된 벡터 데이터를 OpenAI Embeddings 모델을 사용하여 변환
    allow_dangerous_deserialization=True  
).as_retriever(
    search_type="similarity", # 유사한 문서를 검색
    search_kwargs={"k": 8} # 최대 8개의 유사한 문서를 반환
) # as_retriever()를 사용하여 문서 검색 가능하도록 변환

In [8]:
from langchain.chains import RetrievalQA # 검색과 질의응답(QA)을 결합한 랭체인 체인

qa_chain = RetrievalQA.from_chain_type(
    llm=deepseek,
    chain_type="stuff", # 검색된 문서를 합쳐서 하나의 입력(prompt)으로 전달
    retriever=retriever, # 	FAISS 기반의 관련 문서 검색
    chain_type_kwargs=chain_type_kwargs # 프롬프트(PromptTemplate) 적용
)

# 사용자의 질문을 입력하여 검색 및 응답 생성
query = "겨울철 다육이 키우는 방법은?"
response = qa_chain.invoke(query)

print(response)

{'query': '겨울철 다육이 키우는 방법은?', 'result': "<think>\nAlright, the user is asking about how to grow radishes during the winter. They've provided a detailed guide with 8 methods, so I need to focus on the specific winter techniques. \n\nFirst, I remember that in the fall or spring, people often cut and store radish shoots for winter use. That makes sense because it's a way to have fresh produce all year round.\n\nNext, during the winter, the soil can get cold, so watering is crucial but needs to be done carefully. It has to be enough without causing ice, which could damage the roots. I should mention avoiding overwatering and maybe using a slow-release fertilizer to keep the soil moist without letting it become waterlogged.\n\nPlanting in containers seems practical because they are easy to move around and can handle cold weather better than large beds. I'll include tips on planting them directly into the soil, which is also a good method for convenience.\n\nPruning is important too. New gro

In [9]:
query = "겨울철 물은 몇 번 주는 것이 적당해?"
response = qa_chain.invoke(query)

print(response)

{'query': '겨울철 물은 몇 번 주는 것이 적당해?', 'result': '<think>\n好的，现在我要帮用户解答冬季应该给多肉植物浇水的频率问题。首先，我得仔细回忆一下多肉植物的 watering schedule 的一般建议。\n\n根据之前提供的信息，多肉在冬季需要维持一定的湿度和温度，不能让土壤过干也不能冻伤。所以，浇水次数应该减少。通常在冬季，很多植物会停止浇水，或者每周浇水一次是比较常见的做法。\n\n不过，用户可能有不同的情况，比如室内的环境湿度如何，或者他们是否有其他的浇水习惯。考虑到多肉喜欢湿润的环境，但又不希望积水，所以冬季浇水的时间和频率需要特别注意。\n\n根据用户的指导方针，冬季浇水应该非常少，甚至可以说是停止。但有时候在非常干燥的情况下，可能每周一次也是合适的。因此，我建议用户在冬季每周浇水最多一次，并且要确保土壤不干也不冻伤。\n\n最后，表达上要亲切、专业，让用户感到被理解和支持。\n</think>\n\n 겨울철은 mult cutaneous를 주 primary water가 적당합니다. 보통은 한 달에 한 번 정도로浇水를 해주면 됩니다.'}
